In [1]:
pip install python-vlc

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show python-vlc

Name: python-vlc
Version: 3.0.18122
Summary: VLC bindings for python.
Home-page: http://wiki.videolan.org/PythonBinding
Author: Olivier Aubert
Author-email: contact@olivieraubert.net
License: LGPL-2.1+
Location: c:\users\yoon\anaconda3\lib\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
from __future__ import unicode_literals
import yt_dlp as youtube_dl

def my_hook(d):
    if d['status'] == 'finished':
        print('Done downloading, now converting ...')

ydl_opts = {
    'download_archive': 'archive.txt',
    'ignoreerrors': True,
    'nooverwrites': True,
    'format': 'bestvideo[height<=1080]+bestaudio/best[height<=1080]/best',       
    'outtmpl': '저장 경로 템플릿',        
    'noplaylist' : False,       
    'progress_hooks': [my_hook],  
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['http://www.youtube.com/watch?v=Lr9xbHtpAfU'])

[Youtube] Lr9xbHtpAfU: has already been recorded in the archive


In [2]:
pip install network

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz (2.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached websockets-11.0.3-cp310-cp310-win_amd64.whl (124 kB)
  Using cached pycryptodomex-3.18.0-cp35-abi3-win_amd64.whl (1.7 MB)
  Using cached Brotli-1.0.9-cp310-cp310-win_amd64.whl (383 kB)
  Using cached mutagen-1.46.0-py3-none-any.whl (193 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Created wheel for yt-dlp: filename=yt_dlp-2023.3.4-py2.py3-none-any.whl size=2750220 sha256=e741d3430103cd8eff3e260ca1663c1d29cf27c4542b8487a10ede23da7b0cfd
  Stored in 

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\yoon\\anaconda3\\Lib\\site-packages\\~ryptodome\\Cipher\\_pkcs1_decode.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
from __future__ import unicode_literals
import yt_dlp as youtube_dl

def my_hook(d):
    if d['status'] == 'finished':
        print('Done downloading, now converting ...')

ydl_opts = {
    'download_archive': 'archive.txt',
    'ignoreerrors': True,
    'nooverwrites': True,
    'format': 'bestvideo[height<=1080]+bestaudio/best[height<=1080]/best',       
    'outtmpl': '저장 경로 템플릿',        
    'noplaylist' : False,       
    'progress_hooks': [my_hook],  
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['http://www.youtube.com/watch?v=Lr9xbHtpAfU'])

[Youtube] Lr9xbHtpAfU: has already been recorded in the archive


In [ ]:
import os
import sys
import queue
import platform
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtCore import Qt

from PyQt5 import QtWidgets, QtGui, QtCore
import vlc



class MiniPlayer(QtWidgets.QMainWindow):
    """Stripped-down PyQt5-based media player class to sync with "master" video.
    """

    def __init__(self, data_queue, master=None):
        QtWidgets.QMainWindow.__init__(self, master)
        self.setWindowTitle("Mini Player")
        self.statusbar = self.statusBar()
        self.statusbar.showMessage("Ready")

        # Create a basic vlc instance
        self.instance = vlc.Instance()

        self.media = None

        # Create an empty vlc media player
        self.mediaplayer = self.instance.media_player_new()

        self.init_ui()
        self.open_file()

        self.timer = QtCore.QTimer(self)
        self.timer.setInterval(10)
        self.timer.timeout.connect(self.update_ui)

        self.data_queue = data_queue
        self.timer.start()

    def init_ui(self):
        """Set up the user interface
        """
        self.widget = QtWidgets.QWidget(self)
        self.setCentralWidget(self.widget)
        self.setWindowFlags(Qt.WindowStaysOnTopHint | Qt.FramelessWindowHint)

        # In this widget, the video will be drawn
        if platform.system() == "Darwin":  # for MacOS
            self.videoframe = QtWidgets.QMacCocoaViewContainer(0)
        else:
            self.videoframe = QtWidgets.QFrame()

        self.palette = self.videoframe.palette()
        self.palette.setColor(QtGui.QPalette.Window, QtGui.QColor(0, 0, 0))
        self.videoframe.setPalette(self.palette)
        self.videoframe.setAutoFillBackground(True)

        self.vboxlayout = QtWidgets.QVBoxLayout()
        self.vboxlayout.addWidget(self.videoframe)
        self.widget.setLayout(self.vboxlayout)

    def open_file(self):
        """Open a media file in a MediaPlayer
        """
        dialog_txt = "Choose Media File"
        filename = QtWidgets.QFileDialog.getOpenFileName(self, dialog_txt, os.path.expanduser('~'))
        if not filename[0]:
            return

        # getOpenFileName returns a tuple, so use only the actual file name
        self.media = self.instance.media_new(filename[0])

        # Put the media in the media player
        self.mediaplayer.set_media(self.media)

        # Parse the metadata of the file
        self.media.parse()

        # Set the title of the track as the window title
        self.setWindowTitle("{}".format(self.media.get_meta(0)))

        # The media player has to be 'connected' to the QFrame (otherwise the
        # video would be displayed in it's own window). This is platform
        # specific, so we must give the ID of the QFrame (or similar object) to
        # vlc. Different platforms have different functions for this
        if platform.system() == "Linux":  # for Linux using the X Server
            self.mediaplayer.set_xwindow(int(self.videoframe.winId()))
        elif platform.system() == "Windows":  # for Windows
            self.mediaplayer.set_hwnd(int(self.videoframe.winId()))
        elif platform.system() == "Darwin":  # for MacOS
            self.mediaplayer.set_nsobject(int(self.videoframe.winId()))

        # Start playing the video as soon as it loads
        self.mediaplayer.play()

    def update_ui(self):
        self.update_statusbar()
        
        try:
            val = self.data_queue.get(block=False)
        except queue.Empty:
            return

        if val == '<':
            self.mediaplayer.set_rate(self.mediaplayer.get_rate() * 0.5)
            return
        if val == '>':
            self.mediaplayer.set_rate(self.mediaplayer.get_rate() * 2)
            return
        if val == 'P':
            self.mediaplayer.play()
            return
        if val == 'p':
            self.mediaplayer.pause()
            return
        if val == 'S':
            self.mediaplayer.stop()
            return

        val = int(val)
        if val != self.mediaplayer.get_time():
            self.mediaplayer.set_time(val)

    def update_statusbar(self):
        mtime = QtCore.QTime(0, 0, 0, 0)
        time = mtime.addMSecs(self.mediaplayer.get_time())
        self.statusbar.showMessage(time.toString())
        
    def play(self):
        played = 0

        def restart(state):
            nonlocal played
            if state != QMediaPlayer.StoppedState:
                return
            if played == 100:
                return
            played += 1
            self.player.play()


def main():
    """Entry point for our simple vlc player
    """
    app = QtWidgets.QApplication(sys.argv)

    data_queue = queue.Queue()

    player = MiniPlayer(data_queue)
    player.show()
    player.resize(480, 480)


    _ = Client("localhost", 10000, data_queue)
    sys.exit(app.exec_())
  


if __name__ == "__main__":
    main()

